In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.stats import norm
from functools import reduce
from pprint import pprint

### Consolidated Tree Construction

In [6]:

def gini_index(y):
    N = len(y)
    s1 = (y == 1).sum()
    #If all or none are of the same class, return infinite value
    if 0 == s1 or N == s1:
        return np.Inf
    _, counts = np.unique(y, return_counts=True)
    classes_prob = [counts[i]/np.sum(counts) for i in range(len(counts))] #fraction of 
    #Gini Formula: 1- SUMMATION(p^2)
    return np.round(1 - np.sum(np.square(classes_prob)),4)



class TreeNode:
    def __init__(self, depth=1, max_depth=None, min_samples_split = 5):
        self.depth = depth
        self.max_depth = max_depth
        self.impurity = None
        if self.max_depth is not None and self.max_depth < self.depth:
            raise Exception("depth > max_depth")
        self.min_samples_split = min_samples_split
        

    def fit(self, X, Y):
        if (len(Y) < self.min_samples_split or len(set(Y)) == 1):
            self.col = None
            self.split = None
            self.left = None
            self.right = None
            self.prediction = Y[0]

        else:
            #Could be written as: cols = range(X.shape[1])
            D = X.shape[1]
            cols = range(D)
            
            
            min_imp = np.Inf #min_imp = Min impurity
            best_col = None
            best_split = None
            # Finding First Split
            #Find the feature (X col) with the highest possible purity 
            for col in cols:
                imp, split = self.find_split(X, Y, col)
                if imp < min_imp:
                    min_imp = imp
                    best_col = col
                    best_split = split

            #If the most pure split is all of one class then don't split
            if min_imp == np.Inf:
                self.col = None
                self.split = None
                self.left = None
                self.right = None
                self.prediction = np.round(Y.mean())
                return

            self.col = best_col
            self.split = best_split
            self.impurity = min_imp
            #if defined max depth is reached, don't make leaf nodes
            if (self.depth == self.max_depth) or (D == 0):
                self.left = None
                self.right = None
                self.prediction = [
                    np.round(Y[X[:,best_col] < self.split].mean()),
                    np.round(Y[X[:,best_col] >= self.split].mean()),
                ]
            
            #Split up tuples by the split rule and recursively make new nodes until 0 gini index (all nodes are pure)
            else:
                left_idx = (X[:,best_col] < best_split)
                # print(left_idx)
                Xleft = X[left_idx]
                Yleft = Y[left_idx]
                self.left = TreeNode(self.depth + 1, self.max_depth)
                self.left.fit(Xleft, Yleft)
                self.left.impurity = gini_index(Yleft)

                right_idx = (X[:,best_col] >= best_split)
                Xright = X[right_idx]
                Yright = Y[right_idx]
                self.right = TreeNode(self.depth + 1, self.max_depth)
                self.right.fit(Xright, Yright)
                self.right.impurity = gini_index(Yright)
        
  
        
        
                
    #Sort X and Y by sorted(X) indexes. Split rule made on X mean. 
    def find_split(self, X, Y, col):
        x_values = X[:, col] #X Values in the column
        sort_idx = np.argsort(x_values)
        #Sort X and Y index by the sorted X values
        x_values = x_values[sort_idx]
        y_values = Y[sort_idx]
        # Split rule based on mean
        split = np.mean(x_values) 
        imp = self.gini_impurity(x_values, y_values, split)
        return imp, split

    #Returns impurity level of the split 
    def gini_impurity(self, x, y, split):
        y0 = y[x < split]
        y1 = y[x >= split]
        n_samples_0, n_samples_1 = len(y0), len(y1)
        gini_y0, gini_y1 = gini_index(y0), gini_index(y1)
        # print(n_samples_0, n_samples_1)
        split_impurity = (n_samples_0*(gini_y0) + n_samples_1*(gini_y1))/(n_samples_0 + n_samples_1)
        return split_impurity

    #!! Too much nesting. Will try to break it up when refactoring so it's more readable
    def predict_one(self, x): 
        # use "is not None" because 0 means False
        if self.col is not None and self.split is not None:
            feature = x[self.col]
            if feature < self.split:
                if self.left:
                    p = self.left.predict_one(x)
                else:
                    p = self.prediction[0]
            else:
                if self.right:
                    p = self.right.predict_one(x)
                else:
                    p = self.prediction[1]
        else:
            # corresponds to having only 1 prediction
            p = self.prediction
        return p

    
    def predict(self, X):
        N = len(X)
        P = np.zeros(N)
        for i in range(N):
            P[i] = self.predict_one(X[i])
        return P

class DecisionTree:
    def __init__(self, max_depth=None, min_sample_split = 5):
        self.max_depth = max_depth

    def fit(self, X, Y):
        self.root = TreeNode(max_depth=self.max_depth)
        self.root.fit(X, Y)

    def predict(self, X):
        return self.root.predict(X)

    def score(self, X, Y):
        P = self.predict(X)
        return np.mean(P == Y)

    

In [13]:
#uninformative names, change when refactoring. Need to ask Sharania about them 
l1=[]
l3=[]
l=[]
l2 = []
counter = 0
leafnodes = []
predict_leafnode = []
class CTCNode:
    def __init__(self, depth=1, max_depth=None, min_samples_split = 5):
        self.depth = depth
        self.max_depth = max_depth
        if self.max_depth is not None and self.max_depth < self.depth:
            raise Exception("depth > max_depth")
        self.min_samples_split = min_samples_split
        self.mean_ = None 
        self.std_ = None
        self.gauss_prob = None
        self.node_type = None
        cumu_prob_right = None
        Cumu_prob_left = None


    def subsample(self, X, Y, n_samples = 10, split_percent = 0.7):
        if len(set(Y)) == 1:
            return 
        Trees = [DecisionTree(max_depth = 1) for i in range(n_samples)]
        Votes = []
        SplitValues = []
        index = []
        sample = {}
        Counter = 0
        #Make n_samples sub trees (bootstrapping method), then fit each one 
        for Tree in Trees:
            idxs = np.random.choice(X.shape[0], size=int(X.shape[0]*split_percent), replace=True) # Random Sampling 
            Xb = X[idxs]
            Yb = Y[idxs]
            if len(set(Yb)) == 1:
                continue
            Tree.fit(Xb, Yb)
            index.append(idxs)
            key = Tree
            value = idxs
            sample.update({key:value})
            Votes.append(Tree.root.col)
            SplitValues.append(Tree.root.split)
        index = list(np.array(index))
        Votes = np.array(Votes)
        SplitValues = np.array(SplitValues)
        
        #get votes for most important column, prunes empty votes if error is caught
        try:
            cols, counts = np.unique(Votes, return_counts=True)
        except:
            idxs = np.where(Votes != None)
            Votes = Votes[idxs]
            SplitValues = SplitValues[idxs]
            cols, counts = np.unique(Votes, return_counts=True)
        if len(counts) == 0:
            return
        VoteWinner = cols[np.argmax(counts)]
        self.best_col = VoteWinner
        Splits = []
        #Find the split rule of each tree
        for idx,Tree in enumerate(Trees):
            if idx >= len(index):
                break
            idxs = index[idx]
            #print(idxs)
            Xa = X[idxs]
            Ya = Y[idxs]
            x_values = Xa[:, VoteWinner]
            sort_idx = np.argsort(x_values)
            x_values = x_values[sort_idx]
            y_values = Ya[sort_idx]
            # Splitting based on mean
            split = np.mean(x_values)
            Splits.append(split)
        Splits = np.array(Splits)    
        self.col = VoteWinner
        self.mean_ = np.mean(Splits)
        self.std_ = np.std(Splits)
        self.gauss_prob= norm(loc = self.mean_,scale= self.std_).cdf(Splits[0])
    


    def leaf(self, Y):
        self.col = None
        self.split = None
        self.left = None
        self.right = None
        leafnodes.append(reduce((lambda x, y: x * y), l)) #idk what "l" is
        if (len(l2) > 0 ):
            leafnodes.append(reduce((lambda x, y: x * y), l2))
        self.prediction = Y[0]
        
    
    def fit(self, X, Y):
        if (len(Y) < self.min_samples_split or len(set(Y)) == 1):
            self.leaf(Y)
            return
        else:
            D = X.shape[1]
            self.subsample(X, Y)
            #Means node is pure and becomes a leaf
            if (self.mean_ is None or self.std_ is None or self.col is None):
                self.leaf(Y)
                return
            #max depth was reached so no new nodes are made
            if (self.depth == self.max_depth) or (D == 0):
                self.left = None
                self.right = None
                self.prediction = [
                    np.round(Y[X[:,self.col] < self.mean_].mean()),
                    np.round(Y[X[:,self.col] >= self.mean_].mean()),
                ]
            #only works with binary split, should work with categorical as well
            else:
                l.append(self.gauss_prob)
                left_idx = (X[:,self.col] < self.mean_)
                Xleft = X[left_idx]
                Yleft = Y[left_idx]

                self.left = CTCNode(self.depth + 1, self.max_depth)
                self.left.fit(Xleft, Yleft)
                
                

                l2.append(1-self.gauss_prob)
                #print(l2)
                right_idx = (X[:,self.col] >= self.mean_)
                Xright = X[right_idx]
                Yright = Y[right_idx]
                self.right = CTCNode(self.depth + 1, self.max_depth)
                self.right.fit(Xright, Yright)
                

   
    #Has no return for most likely circumstance
    def predict_one(self, x):
        # use "is not None" because 0 means False
        if self.col is not None and self.mean_ is not None:
            feature = x[self.col]
            #recursively move down the tree until you find the leaf
            if feature < self.mean_:
                if self.left: 
                    l1.append(self.gauss_prob)
                    cumu_gauss_left = reduce((lambda x, y: x * y), l1)
                    p = self.left.predict_one(x)
                if self.leaf:

                    predict_leafnode.append(cumu_gauss_left)
                    
                else:
                    p = self.prediction[0]
                    
            else:
                if self.right:
                    l3.append(1-self.gauss_prob)
                    cumu_gauss_right = reduce((lambda x, y: x * y), l3)
                    p = self.right.predict_one(x)
                if self.leaf:
                    predict_leafnode.append(cumu_gauss_right)
                else:
                    p = self.prediction[1]
           
        else:
            # corresponds to having only 1 prediction
            p = self.prediction   
        return p

    def predict(self, X):
        N = len(X)
        P = np.zeros(N)
        for i in range(N):
            P[i] = self.predict_one(X[i])
        return P

    
     

class CTC_DecisionTree:
    def __init__(self, max_depth=None, min_sample_split = 5):
        self.root = None
        self.max_depth = max_depth

    def fit(self, X, Y):
        self.root = CTCNode(max_depth=self.max_depth)
        self.root.fit(X, Y)
       


    def predict(self, X):
        return self.root.predict(X)

    def score(self, X, Y):
        P = self.predict(X)
        return np.mean(P == Y)

In [ ]:
#Running just like scikit learn
# model = CTC_DecisionTree(max_depth=13)
# model.fit(Xtrain, Ytrain)

#y_pred = model.predict(Xtest)

# model.score(Xtest,Ytest) 